In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

## Custom Factors
When we first looked at factors, we explored the set of built-in factors. Frequently, a desired computation isn't included as a built-in factor. One of the most powerful features of the Pipeline API is that it allows us to define our own custom factors. When a desired computation doesn't exist as a built-in, we define a custom factor.

Conceptually, a custom factor is identical to a built-in factor. It accepts `inputs`, `window_length`, and `mask` as constructor arguments, and returns a `Factor` object each day.

Let's take an example of a computation that doesn't exist as a built-in: standard deviation. To create a factor that computes the [standard deviation](https://en.wikipedia.org/wiki/Standard_deviation) over a trailing window, we can subclass `quantopian.pipeline.CustomFactor` and implement a compute method whose signature is:


```
def compute(self, today, asset_ids, out, *inputs):
    ...
```

- `*inputs` are M x N [numpy arrays](http://docs.scipy.org/doc/numpy-1.10.1/reference/arrays.ndarray.html), where M is the `window_length` and N is the number of securities (usually around ~8000 unless a `mask` is provided). `*inputs` are trailing data windows. Note that there will be one M x N array for each `BoundColumn` provided in the factor's `inputs` list. The data type of each array will be the `dtype` of the corresponding `BoundColumn`.
- `out` is an empty array of length N. `out` will be the output of our custom factor each day. The job of compute is to write output values into out.
- `asset_ids` will be an integer [array](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.array.html) of length N containing security ids corresponding to the columns in our `*inputs` arrays.
- `today` will be a [pandas Timestamp](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#converting-to-timestamps) representing the day for which `compute` is being called.

Of these, `*inputs` and `out` are most commonly used.

An instance of `CustomFactor` that’s been added to a pipeline will have its compute method called every day. For example, let's define a custom factor that computes the standard deviation of the close price over the last 5 days. To start, let's add `CustomFactor` and `numpy` to our import statements.

In [2]:
from quantopian.pipeline import CustomFactor
import numpy

Next, let's define our custom factor to calculate the standard deviation over a trailing window using [numpy.nanstd](http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.nanstd.html):

In [3]:
class StdDev(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        # Calculates the column-wise standard deviation, ignoring NaNs
        out[:] = numpy.nanstd(values, axis=0)

Finally, let's instantiate our factor in `make_pipeline()`:

In [4]:
def make_pipeline():
    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev
        }
    )

When this pipeline is run, `StdDev.compute()` will be called every day with data as follows:

- `values`: An M x N [numpy array](http://docs.scipy.org/doc/numpy-1.10.1/reference/arrays.ndarray.html), where M is 5 (`window_length`), and N is ~8000 (the number of securities in our database on the day in question).
- `out`: An empty array of length N (~8000). In this example, the job of `compute` is to populate `out` with an array storing of 5-day close price standard deviations.

In [6]:
result = run_pipeline(make_pipeline(), '2019-05-05', '2019-05-05')
result

std_dev
2019-05-06 00:00:00+00:00 Equity(2 [HWM])          0.569634
                          Equity(21 [AAME])        0.064686
                          Equity(24 [AAPL])        4.158370
                          Equity(25 [HWM_PR])      0.546008
                          Equity(41 [ARCB])        0.282800
                          Equity(52 [ABM])         0.592182
                          Equity(53 [ABMD])        4.595548
                          Equity(62 [ABT])         0.420314
                          Equity(64 [GOLD])        0.071944
                          Equity(66 [AB])          0.189752
                          Equity(67 [ADSK])        1.847535
                          Equity(70 [VBF])         0.067409
                          Equity(76 [TAP])         1.966534
                          Equity(100 [IEP])        0.482634
                          Equity(106 [ACU])        0.729544
                          Equity(110 [RAMP])       0.495072
                          Equity(112 [ACY])        0.176989
                          Equity(114 [ADBE])       3.237125
                          Equity(117 [AEY])        0.016000
                          Equity(122 [ADI])        1.251904
                          Equity(128 [ADM])        0.457410
                          Equity(149 [ADX])        0.050754
                          Equity(153 [AE])         0.613893
                          Equity(154 [AEM])        0.393375
                          Equity(157 [AEG])        0.027641
                          Equity(161 [AEP])        0.532939
                          Equity(166 [AES])        0.087825
                          Equity(185 [AFL])        0.267462
                          Equity(192 [ATAX])       0.064684
                          Equity(197 [AGCO])       2.118522
...                                                     ...
                          Equity(53097 [GNLN])     1.585612
                          Equity(53098 [SHLL])     0.000000
                          Equity(53099 [RYLD])     0.063364
                          Equity(53100 [SHLL_WS])  0.000000
                          Equity(53101 [MREO])     0.316158
                          Equity(53102 [SOHO_N])   0.055130
                          Equity(53103 [USI])      0.000000
                          Equity(53104 [ALTG_WS])  0.015000
                          Equity(53105 [SEIX])     0.010000
                          Equity(53106 [ALTG])     0.000000
                          Equity(53107 [ACTT_U])   0.005044
                          Equity(53108 [ETP_PRE])  0.024875
                          Equity(53109 [WTRU])     0.349544
                          Equity(53112 [WAFU])     0.014142
                          Equity(53113 [TPLC])     0.152500
                          Equity(53114 [TPHD])     0.139500
                          Equity(53115 [SY])       0.000000
                          Equity(53116 [BYND])     0.000000
                          Equity(53117 [TRVI])          NaN
                          Equity(53118 [TMDX])     0.000000
                          Equity(53119 [KEY_PRK])       NaN
                          Equity(53120 [YJ])            NaN
                          Equity(53121 [BUL])           NaN
                          Equity(53122 [RRBI])          NaN
                          Equity(53123 [SCPL])          NaN
                          Equity(53124 [TRNE_WS])       NaN
                          Equity(53125 [TRNE])          NaN
                          Equity(53126 [PTIN])          NaN
                          Equity(53127 [ATIF])          NaN
                          Equity(53128 [ECOW])          NaN

[8733 rows x 1 columns]

### Default Inputs
When writing a custom factor, we can set default `inputs` and `window_length` in our `CustomFactor` subclass. For example, let's define the `TenDayMeanDifference` custom factor to compute the mean difference between two data columns over a trailing window using [numpy.nanmean](http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.nanmean.html). Let's set the default `inputs` to `[USEquityPricing.close, USEquityPricing.open]` and the default `window_length` to 10:

In [7]:
class TenDayMeanDifference(CustomFactor):
    # Default inputs.
    inputs = [USEquityPricing.close, USEquityPricing.open]
    window_length = 10
    def compute(self, today, asset_ids, out, close, open):
        # Calculates the column-wise mean difference, ignoring NaNs
        out[:] = numpy.nanmean(close - open, axis=0)

<i>Remember in this case that `close` and `open` are each 10 x ~8000 2D [numpy arrays.](http://docs.scipy.org/doc/numpy-1.10.1/reference/arrays.ndarray.html)</i>

If we call `TenDayMeanDifference` without providing any arguments, it will use the defaults.

In [8]:
# Computes the 10-day mean difference between the daily open and close prices.
close_open_diff = TenDayMeanDifference()

The defaults can be manually overridden by specifying arguments in the constructor call.

In [9]:
# Computes the 10-day mean difference between the daily high and low prices.
high_low_diff = TenDayMeanDifference(inputs=[USEquityPricing.high, USEquityPricing.low])

### Further Example
Let's take another example where we build a [momentum](http://www.investopedia.com/terms/m/momentum.asp) custom factor and use it to create a filter. We will then use that filter as a `screen` for our pipeline.

Let's start by defining a `Momentum` factor to be the division of the most recent close price by the close price from `n` days ago where `n` is the `window_length`.

In [10]:
class Momentum(CustomFactor):
    # Default inputs
    inputs = [USEquityPricing.close]

    # Compute momentum
    def compute(self, today, assets, out, close):
        out[:] = close[-1] / close[0]

Now, let's instantiate our `Momentum` factor (twice) to create a 10-day momentum factor and a 20-day momentum factor. Let's also create a `positive_momentum` filter returning `True` for securities with both a positive 10-day momentum and a positive 20-day momentum.

In [11]:
ten_day_momentum = Momentum(window_length=10)
twenty_day_momentum = Momentum(window_length=20)

positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

Next, let's add our momentum factors and our `positive_momentum` filter to `make_pipeline`. Let's also pass `positive_momentum` as a `screen` to our pipeline.

In [12]:
def make_pipeline():

    ten_day_momentum = Momentum(window_length=10)
    twenty_day_momentum = Momentum(window_length=20)

    positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev,
            'ten_day_momentum': ten_day_momentum,
            'twenty_day_momentum': twenty_day_momentum
        },
        screen=positive_momentum
    )

Running this pipeline outputs the standard deviation and each of our momentum computations for securities with positive 10-day and 20-day momentum.

In [ ]:
result = run_pipeline(make_pipeline(), '2019-05-05', '2019-05-05')
result

Custom factors allow us to define custom computations in a pipeline. They are frequently the best way to perform computations on [partner datasets](https://www.quantopian.com/data) or on multiple data columns. The full documentation for CustomFactors is available [here](https://www.quantopian.com/help#custom-factors).

In the next lesson, we'll use everything we've learned so far to create a pipeline for an algorithm.